# Chapter 18: Testing and Evaluation
**From: Zero to AI Agent**

## Overview
In this chapter, you'll learn about:
- Unit testing agent components
- Integration testing strategies
- Evaluating agent performance
- Creating test datasets
- Measuring accuracy and reliability
- A/B testing agents
- Continuous improvement workflows


In [ ]:
!pip install -q -r requirements.txt

from dotenv import load_dotenv
load_dotenv()

---
## Section 18.1: Unit testing agent components

In [ ]:
# From: test_example.py

# From: AI Agents Book, Chapter 18, Section 18.1
# File: test_example.py
# Description: Basic pytest examples to get started with testing

def test_addition():
    """Our very first test!"""
    result = 2 + 2
    assert result == 4


def test_string_contains():
    """Test that a string contains expected text."""
    message = "Hello, World!"
    assert "World" in message


def test_list_length():
    """Test the length of a list."""
    items = [1, 2, 3, 4, 5]
    assert len(items) == 5


def test_that_fails():
    """This test will fail - on purpose! 
    Uncomment to see what failure output looks like."""
    # result = 2 + 2
    # assert result == 5, "Math is broken!"
    pass


In [ ]:
# From: tools.py

# From: AI Agents Book, Chapter 18, Section 18.1
# File: tools.py
# Description: Agent tool functions - shipping calculator and customer lookup

import httpx


def calculate_shipping(weight_kg: float, distance_km: float, express: bool = False) -> dict:
    """Calculate shipping cost based on weight and distance."""
    base_rate = 5.00
    weight_charge = weight_kg * 0.50
    distance_charge = distance_km * 0.01
    
    subtotal = base_rate + weight_charge + distance_charge
    
    if express:
        subtotal *= 1.5
    
    return {
        "cost": round(subtotal, 2),
        "currency": "USD",
        "express": express,
        "estimated_days": 1 if express else 5
    }


async def get_customer_info(customer_id: str) -> dict:
    """Fetch customer information from the CRM API."""
    async with httpx.AsyncClient() as client:
        response = await client.get(
            f"https://api.example.com/customers/{customer_id}",
            headers={"Authorization": "Bearer secret-token"}
        )
        response.raise_for_status()
        return response.json()


In [ ]:
# From: test_tools.py

# From: AI Agents Book, Chapter 18, Section 18.1
# File: test_tools.py
# Description: Unit tests for agent tools - demonstrates mocking external services

import pytest
from unittest.mock import AsyncMock, Mock, patch
from tools import calculate_shipping, get_customer_info


# Tests for calculate_shipping (deterministic - no mocking needed)

def test_basic_shipping_calculation():
    result = calculate_shipping(weight_kg=2.0, distance_km=100)
    
    assert result["cost"] == 7.00  # 5 + (2 * 0.5) + (100 * 0.01)
    assert result["currency"] == "USD"
    assert result["express"] is False
    assert result["estimated_days"] == 5


def test_express_shipping_multiplier():
    standard = calculate_shipping(weight_kg=2.0, distance_km=100, express=False)
    express = calculate_shipping(weight_kg=2.0, distance_km=100, express=True)
    
    assert express["cost"] == standard["cost"] * 1.5
    assert express["estimated_days"] == 1


def test_zero_weight_and_distance():
    result = calculate_shipping(weight_kg=0, distance_km=0)
    
    assert result["cost"] == 5.00  # Just the base rate


# Tests for get_customer_info (requires mocking the HTTP client)

@pytest.mark.asyncio
async def test_get_customer_info_success():
    mock_response = {
        "id": "cust_123",
        "name": "Alice Smith",
        "email": "alice@example.com",
        "tier": "premium"
    }
    
    with patch("tools.httpx.AsyncClient") as MockClient:
        # Set up the mock to return our fake response
        # Use regular Mock for response since json() is sync in httpx
        mock_response_obj = Mock()
        mock_response_obj.json.return_value = mock_response
        mock_response_obj.raise_for_status = Mock()

        mock_client_instance = AsyncMock()
        mock_client_instance.get.return_value = mock_response_obj
        MockClient.return_value.__aenter__.return_value = mock_client_instance
        
        result = await get_customer_info("cust_123")
        
        assert result["name"] == "Alice Smith"
        assert result["tier"] == "premium"
        
        # Verify the API was called correctly
        mock_client_instance.get.assert_called_once()
        call_args = mock_client_instance.get.call_args
        assert "cust_123" in call_args[0][0]


In [ ]:
# From: state.py

# From: AI Agents Book, Chapter 18, Section 18.1
# File: state.py
# Description: Agent state definitions and helper functions for state transformations

from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import add_messages


class AgentState(TypedDict):
    messages: Annotated[list, add_messages]
    current_tool: str | None
    retry_count: int


def should_retry(state: AgentState, max_retries: int = 3) -> bool:
    """Determine if we should retry the current operation."""
    return state["retry_count"] < max_retries


def increment_retry(state: AgentState) -> dict:
    """Return state update to increment retry count."""
    return {"retry_count": state["retry_count"] + 1}


def reset_retry(state: AgentState) -> dict:
    """Return state update to reset retry count."""
    return {"retry_count": 0}


In [ ]:
# From: test_state.py

# From: AI Agents Book, Chapter 18, Section 18.1
# File: test_state.py
# Description: Unit tests for state transformation functions

from state import should_retry, increment_retry, reset_retry


def test_should_retry_under_limit():
    state = {"messages": [], "current_tool": None, "retry_count": 1}
    assert should_retry(state, max_retries=3) is True


def test_should_retry_at_limit():
    state = {"messages": [], "current_tool": None, "retry_count": 3}
    assert should_retry(state, max_retries=3) is False


def test_increment_retry():
    state = {"messages": [], "current_tool": None, "retry_count": 2}
    update = increment_retry(state)
    
    assert update == {"retry_count": 3}


def test_reset_retry():
    state = {"messages": [], "current_tool": None, "retry_count": 5}
    update = reset_retry(state)
    
    assert update == {"retry_count": 0}


In [ ]:
# From: routing.py

# From: AI Agents Book, Chapter 18, Section 18.1
# File: routing.py
# Description: Conditional edge routing logic for LangGraph agents


def route_after_llm(state: dict) -> str:
    """Decide where to go after the LLM responds."""
    messages = state.get("messages", [])
    
    if not messages:
        return "error_handler"
    
    last_message = messages[-1]
    
    # Check if LLM wants to use a tool
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "execute_tools"
    
    # Check if conversation should end
    if state.get("should_end", False):
        return "goodbye"
    
    # Continue conversation
    return "respond_to_user"


In [ ]:
# From: test_routing.py

# From: AI Agents Book, Chapter 18, Section 18.1
# File: test_routing.py
# Description: Unit tests for conditional edge routing decisions

from unittest.mock import MagicMock
from routing import route_after_llm


def test_route_to_tools_when_tool_calls_present():
    mock_message = MagicMock()
    mock_message.tool_calls = [{"name": "search", "args": {"query": "test"}}]
    
    state = {"messages": [mock_message]}
    
    assert route_after_llm(state) == "execute_tools"


def test_route_to_goodbye_when_should_end():
    mock_message = MagicMock()
    mock_message.tool_calls = None
    
    state = {"messages": [mock_message], "should_end": True}
    
    assert route_after_llm(state) == "goodbye"


def test_route_to_respond_by_default():
    mock_message = MagicMock()
    mock_message.tool_calls = None
    
    state = {"messages": [mock_message], "should_end": False}
    
    assert route_after_llm(state) == "respond_to_user"


def test_route_to_error_when_no_messages():
    state = {"messages": []}
    
    assert route_after_llm(state) == "error_handler"


In [ ]:
# From: nodes.py

# From: AI Agents Book, Chapter 18, Section 18.1
# File: nodes.py
# Description: Agent node functions - demonstrates LLM integration pattern

from langchain_openai import ChatOpenAI


async def analyze_sentiment(state: dict, llm: ChatOpenAI) -> dict:
    """Analyze the sentiment of the user's last message."""
    messages = state["messages"]
    last_user_message = None
    
    for msg in reversed(messages):
        if msg.type == "human":
            last_user_message = msg.content
            break
    
    if not last_user_message:
        return {"sentiment": "unknown", "confidence": 0.0}
    
    response = await llm.ainvoke([
        {"role": "system", "content": "Analyze sentiment. Respond with only: positive, negative, or neutral"},
        {"role": "user", "content": last_user_message}
    ])
    
    sentiment = response.content.strip().lower()
    
    # Validate the response
    if sentiment not in ["positive", "negative", "neutral"]:
        sentiment = "unknown"
    
    return {"sentiment": sentiment, "confidence": 0.85}


In [ ]:
# From: test_nodes.py

# From: AI Agents Book, Chapter 18, Section 18.1
# File: test_nodes.py
# Description: Unit tests for node functions - demonstrates mocking LLM responses

import pytest
from unittest.mock import AsyncMock, MagicMock
from nodes import analyze_sentiment


@pytest.mark.asyncio
async def test_analyze_sentiment_positive():
    # Create mock LLM
    mock_llm = AsyncMock()
    mock_response = MagicMock()
    mock_response.content = "positive"
    mock_llm.ainvoke.return_value = mock_response
    
    # Create test state
    mock_message = MagicMock()
    mock_message.type = "human"
    mock_message.content = "I love this product!"
    state = {"messages": [mock_message]}
    
    result = await analyze_sentiment(state, mock_llm)
    
    assert result["sentiment"] == "positive"
    assert result["confidence"] == 0.85


@pytest.mark.asyncio
async def test_analyze_sentiment_handles_invalid_response():
    mock_llm = AsyncMock()
    mock_response = MagicMock()
    mock_response.content = "I think it's somewhat positive but also..."  # Invalid
    mock_llm.ainvoke.return_value = mock_response
    
    mock_message = MagicMock()
    mock_message.type = "human"
    mock_message.content = "It's okay I guess"
    state = {"messages": [mock_message]}
    
    result = await analyze_sentiment(state, mock_llm)
    
    assert result["sentiment"] == "unknown"  # Falls back gracefully


@pytest.mark.asyncio
async def test_analyze_sentiment_no_user_message():
    mock_llm = AsyncMock()
    
    state = {"messages": []}  # No messages
    
    result = await analyze_sentiment(state, mock_llm)
    
    assert result["sentiment"] == "unknown"
    assert result["confidence"] == 0.0
    mock_llm.ainvoke.assert_not_called()  # LLM shouldn't be called


---
### Section 18.1 Exercises

### Exercise 18.1.1: Testing a Validation Tool

Create a tool function called `validate_email` that checks if an email address is valid. It should return a dictionary with `valid` (boolean), `error` (string or None), and `normalized` (the email in lowercase). Write at least 5 unit tests covering valid emails, invalid emails, edge cases like empty strings, and emails with unusual but valid characters.

In [ ]:
# Your code here


### Exercise 18.1.2: Testing State Helpers

Create a set of helper functions for managing a "shopping cart" state that includes `items` (list of dicts with name and quantity), `total` (float), and `coupon_code` (string or None). Write functions for `add_item`, `remove_item`, `apply_coupon`, and `calculate_total`. Write unit tests for each function, including edge cases like removing an item that doesn't exist or applying an invalid coupon.

In [ ]:
# Your code here


### Exercise 18.1.3: Mocking an API Tool

You have a tool that calls a weather API to get the current temperature for a city. Write the tool function and then write tests using mocks that verify:
- The function correctly parses a successful API response
- The function handles a 404 (city not found) gracefully
- The function handles network timeouts gracefully
- The correct city name is passed to the API

In [ ]:
# Your code here


---
## Section 18.2: Integration testing strategies

In [ ]:
# From: support_agent.py

# From: AI Agents Book, Chapter 18, Section 18.2
# File: support_agent.py
# Description: Example support agent with classification and routing

from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage


class SupportState(TypedDict):
    messages: Annotated[list, add_messages]
    issue_type: str | None
    resolved: bool


def classify_issue(state: SupportState) -> dict:
    """Classify the customer's issue based on their message."""
    last_message = state["messages"][-1].content.lower()
    
    if "refund" in last_message or "money back" in last_message:
        return {"issue_type": "refund"}
    elif "broken" in last_message or "not working" in last_message:
        return {"issue_type": "technical"}
    elif "cancel" in last_message:
        return {"issue_type": "cancellation"}
    else:
        return {"issue_type": "general"}


def route_by_issue(state: SupportState) -> str:
    """Route to appropriate handler based on issue type."""
    issue_type = state.get("issue_type")
    if issue_type == "refund":
        return "handle_refund"
    elif issue_type == "technical":
        return "handle_technical"
    elif issue_type == "cancellation":
        return "handle_cancellation"
    else:
        return "handle_general"


def handle_refund(state: SupportState, llm) -> dict:
    """Handle refund requests."""
    response = llm.invoke(state["messages"] + [
        HumanMessage(content="Generate a helpful response about our refund policy.")
    ])
    return {
        "messages": [AIMessage(content=response.content)],
        "resolved": True
    }


def handle_technical(state: SupportState, llm) -> dict:
    """Handle technical issues."""
    response = llm.invoke(state["messages"] + [
        HumanMessage(content="Generate a helpful response for technical support.")
    ])
    return {
        "messages": [AIMessage(content=response.content)],
        "resolved": True
    }


def handle_cancellation(state: SupportState, llm) -> dict:
    """Handle cancellation requests."""
    response = llm.invoke(state["messages"] + [
        HumanMessage(content="Generate a helpful response about cancellation.")
    ])
    return {
        "messages": [AIMessage(content=response.content)],
        "resolved": True
    }


def handle_general(state: SupportState, llm) -> dict:
    """Handle general inquiries."""
    response = llm.invoke(state["messages"] + [
        HumanMessage(content="Generate a helpful general response.")
    ])
    return {
        "messages": [AIMessage(content=response.content)],
        "resolved": True
    }


def build_support_graph(llm):
    """Build the support agent graph."""
    graph = StateGraph(SupportState)
    
    graph.add_node("classify", classify_issue)
    graph.add_node("handle_refund", lambda s: handle_refund(s, llm))
    graph.add_node("handle_technical", lambda s: handle_technical(s, llm))
    graph.add_node("handle_cancellation", lambda s: handle_cancellation(s, llm))
    graph.add_node("handle_general", lambda s: handle_general(s, llm))
    
    graph.add_edge(START, "classify")
    graph.add_conditional_edges("classify", route_by_issue)
    graph.add_edge("handle_refund", END)
    graph.add_edge("handle_technical", END)
    graph.add_edge("handle_cancellation", END)
    graph.add_edge("handle_general", END)
    
    return graph.compile()


In [ ]:
# From: test_support_agent.py

# From: AI Agents Book, Chapter 18, Section 18.2
# File: test_support_agent.py
# Description: Integration tests demonstrating various testing strategies

import pytest
import os
from unittest.mock import MagicMock
from langchain_core.messages import HumanMessage, AIMessage
from support_agent import build_support_graph, SupportState


@pytest.fixture
def mock_llm():
    """Create a mock LLM that returns predictable responses."""
    llm = MagicMock()
    llm.invoke.return_value = MagicMock(
        content="I'd be happy to help you with that!"
    )
    return llm


# --- Basic Routing Tests ---

def test_refund_request_routes_correctly(mock_llm):
    """Test that refund requests go to the refund handler."""
    graph = build_support_graph(mock_llm)
    
    initial_state = {
        "messages": [HumanMessage(content="I want a refund for my order")],
        "issue_type": None,
        "resolved": False
    }
    
    result = graph.invoke(initial_state)
    
    # Verify routing worked correctly
    assert result["issue_type"] == "refund"
    assert result["resolved"] is True
    
    # Verify LLM was called (handler executed)
    assert mock_llm.invoke.called


def test_technical_issue_routes_correctly(mock_llm):
    """Test that technical issues go to the technical handler."""
    graph = build_support_graph(mock_llm)
    
    initial_state = {
        "messages": [HumanMessage(content="My device is broken")],
        "issue_type": None,
        "resolved": False
    }
    
    result = graph.invoke(initial_state)
    
    assert result["issue_type"] == "technical"
    assert result["resolved"] is True


# --- State Transition Tests ---

def test_state_accumulates_through_workflow(mock_llm):
    """Verify state is correctly passed and accumulated."""
    graph = build_support_graph(mock_llm)
    
    initial_state = {
        "messages": [HumanMessage(content="I need a refund please")],
        "issue_type": None,
        "resolved": False
    }
    
    result = graph.invoke(initial_state)
    
    # Original message should still be present
    assert len(result["messages"]) >= 1
    assert result["messages"][0].content == "I need a refund please"
    
    # AI response should have been added
    ai_messages = [m for m in result["messages"] if isinstance(m, AIMessage)]
    assert len(ai_messages) >= 1
    
    # State fields should be populated
    assert result["issue_type"] is not None
    assert result["resolved"] is True


# --- Property-Based Tests ---

def test_agent_always_responds(mock_llm):
    """Verify the agent always produces a response."""
    graph = build_support_graph(mock_llm)
    
    test_messages = [
        "I want a refund",
        "This is broken",
        "Cancel my subscription",
        "Hello, I have a question",
        "asdfghjkl",  # Gibberish input
    ]
    
    for message in test_messages:
        initial_state = {
            "messages": [HumanMessage(content=message)],
            "issue_type": None,
            "resolved": False
        }
        
        result = graph.invoke(initial_state)
        
        # Property: Agent should always add a response
        assert len(result["messages"]) > 1, f"No response for: {message}"
        
        # Property: Workflow should complete
        assert result["resolved"] is True, f"Not resolved for: {message}"


def test_agent_never_loses_messages(mock_llm):
    """Verify messages are never dropped during processing."""
    graph = build_support_graph(mock_llm)
    
    initial_messages = [
        HumanMessage(content="First message"),
        AIMessage(content="First response"),
        HumanMessage(content="I want a refund now"),
    ]
    
    initial_state = {
        "messages": initial_messages,
        "issue_type": None,
        "resolved": False
    }
    
    result = graph.invoke(initial_state)
    
    # All original messages should still be present
    assert len(result["messages"]) >= len(initial_messages)
    
    # Check that original content is preserved
    result_contents = [m.content for m in result["messages"]]
    for original in initial_messages:
        assert original.content in result_contents


# --- Error Handling Tests ---

def test_llm_error_is_handled_gracefully():
    """Test that LLM errors don't crash the entire workflow."""
    # Create an LLM that raises an error
    failing_llm = MagicMock()
    failing_llm.invoke.side_effect = Exception("API rate limit exceeded")
    
    graph = build_support_graph(failing_llm)
    
    initial_state = {
        "messages": [HumanMessage(content="I need help")],
        "issue_type": None,
        "resolved": False
    }
    
    # Expect the exception to propagate (no built-in error handling)
    with pytest.raises(Exception) as exc_info:
        graph.invoke(initial_state)
    
    assert "rate limit" in str(exc_info.value).lower()


# --- Integration Test with Real LLM (run sparingly) ---

@pytest.mark.integration
@pytest.mark.skipif(
    not os.getenv("OPENAI_API_KEY"),
    reason="No API key available"
)
def test_with_real_llm():
    """Integration test with actual LLM - run sparingly."""
    from langchain_openai import ChatOpenAI
    
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
    graph = build_support_graph(llm)
    
    initial_state = {
        "messages": [HumanMessage(content="I want to return my order #12345")],
        "issue_type": None,
        "resolved": False
    }
    
    result = graph.invoke(initial_state)

    # Test properties, not exact content
    # Note: We accept multiple issue types because LLM classification is
    # non-deterministic. Even with temperature=0, the model may classify
    # "return my order" as "refund", "general", or "order" depending on
    # subtle variations in model behavior across API calls.
    assert result["issue_type"] in ["refund", "general", "order"]
    assert result["resolved"] is True
    
    # Check that response mentions relevant information
    response = result["messages"][-1].content.lower()
    assert any(word in response for word in ["refund", "return", "order", "help"])


In [ ]:
# From: conftest.py

# From: AI Agents Book, Chapter 18, Section 18.2
# File: conftest.py
# Description: Shared pytest fixtures for integration tests

import os
import pytest
from unittest.mock import MagicMock
from dotenv import load_dotenv

# Load .env file for tests that need API keys
load_dotenv()


@pytest.fixture
def mock_llm():
    """Standard mock LLM for integration tests."""
    llm = MagicMock()
    llm.invoke.return_value = MagicMock(content="Mock response")
    return llm


@pytest.fixture
def configured_mock_llm():
    """Mock LLM that can be configured with specific responses."""
    def _create_mock(responses: list[str]):
        llm = MagicMock()
        llm.invoke.side_effect = [
            MagicMock(content=r) for r in responses
        ]
        return llm
    return _create_mock


In [ ]:
# From: test_tools.py

# From: AI Agents Book, Chapter 18, Section 18.1
# File: test_tools.py
# Description: Unit tests for agent tools - demonstrates mocking external services

import pytest
from unittest.mock import AsyncMock, Mock, patch
from tools import calculate_shipping, get_customer_info


# Tests for calculate_shipping (deterministic - no mocking needed)

def test_basic_shipping_calculation():
    result = calculate_shipping(weight_kg=2.0, distance_km=100)
    
    assert result["cost"] == 7.00  # 5 + (2 * 0.5) + (100 * 0.01)
    assert result["currency"] == "USD"
    assert result["express"] is False
    assert result["estimated_days"] == 5


def test_express_shipping_multiplier():
    standard = calculate_shipping(weight_kg=2.0, distance_km=100, express=False)
    express = calculate_shipping(weight_kg=2.0, distance_km=100, express=True)
    
    assert express["cost"] == standard["cost"] * 1.5
    assert express["estimated_days"] == 1


def test_zero_weight_and_distance():
    result = calculate_shipping(weight_kg=0, distance_km=0)
    
    assert result["cost"] == 5.00  # Just the base rate


# Tests for get_customer_info (requires mocking the HTTP client)

@pytest.mark.asyncio
async def test_get_customer_info_success():
    mock_response = {
        "id": "cust_123",
        "name": "Alice Smith",
        "email": "alice@example.com",
        "tier": "premium"
    }
    
    with patch("tools.httpx.AsyncClient") as MockClient:
        # Set up the mock to return our fake response
        # Use regular Mock for response since json() is sync in httpx
        mock_response_obj = Mock()
        mock_response_obj.json.return_value = mock_response
        mock_response_obj.raise_for_status = Mock()

        mock_client_instance = AsyncMock()
        mock_client_instance.get.return_value = mock_response_obj
        MockClient.return_value.__aenter__.return_value = mock_client_instance
        
        result = await get_customer_info("cust_123")
        
        assert result["name"] == "Alice Smith"
        assert result["tier"] == "premium"
        
        # Verify the API was called correctly
        mock_client_instance.get.assert_called_once()
        call_args = mock_client_instance.get.call_args
        assert "cust_123" in call_args[0][0]


In [ ]:
# From: test_state.py

# From: AI Agents Book, Chapter 18, Section 18.1
# File: test_state.py
# Description: Unit tests for state transformation functions

from state import should_retry, increment_retry, reset_retry


def test_should_retry_under_limit():
    state = {"messages": [], "current_tool": None, "retry_count": 1}
    assert should_retry(state, max_retries=3) is True


def test_should_retry_at_limit():
    state = {"messages": [], "current_tool": None, "retry_count": 3}
    assert should_retry(state, max_retries=3) is False


def test_increment_retry():
    state = {"messages": [], "current_tool": None, "retry_count": 2}
    update = increment_retry(state)
    
    assert update == {"retry_count": 3}


def test_reset_retry():
    state = {"messages": [], "current_tool": None, "retry_count": 5}
    update = reset_retry(state)
    
    assert update == {"retry_count": 0}


In [ ]:
# From: test_routing.py

# From: AI Agents Book, Chapter 18, Section 18.1
# File: test_routing.py
# Description: Unit tests for conditional edge routing decisions

from unittest.mock import MagicMock
from routing import route_after_llm


def test_route_to_tools_when_tool_calls_present():
    mock_message = MagicMock()
    mock_message.tool_calls = [{"name": "search", "args": {"query": "test"}}]
    
    state = {"messages": [mock_message]}
    
    assert route_after_llm(state) == "execute_tools"


def test_route_to_goodbye_when_should_end():
    mock_message = MagicMock()
    mock_message.tool_calls = None
    
    state = {"messages": [mock_message], "should_end": True}
    
    assert route_after_llm(state) == "goodbye"


def test_route_to_respond_by_default():
    mock_message = MagicMock()
    mock_message.tool_calls = None
    
    state = {"messages": [mock_message], "should_end": False}
    
    assert route_after_llm(state) == "respond_to_user"


def test_route_to_error_when_no_messages():
    state = {"messages": []}
    
    assert route_after_llm(state) == "error_handler"


---
### Section 18.2 Exercises

### Exercise 18.2.1: Testing a Multi-Node Workflow

Create a simple order processing graph with three nodes: `validate_order` (checks if order data is valid), `calculate_total` (computes total with tax), and `confirm_order` (generates confirmation). Write integration tests that verify: (1) valid orders flow through all three nodes, (2) invalid orders stop at validation, and (3) state accumulates correctly through the workflow.

In [ ]:
# Your code here


### Exercise 18.2.2: Testing Branching Logic

Build a graph that routes customer messages to different handlers based on sentiment (positive, negative, neutral). The routing should happen after a classification node. Write integration tests that verify each branch is taken for appropriate inputs, and that the correct handler is invoked. Use mocks to make tests deterministic.

In [ ]:
# Your code here


### Exercise 18.2.3: Testing Conversation Context

Create a simple FAQ agent that answers questions differently based on conversation history. For example, if the user already asked about pricing, a follow-up question like "what about discounts?" should be understood in context. Write integration tests that verify context is maintained across turns and influences responses appropriately.

In [ ]:
# Your code here


---
## Section 18.3: Evaluating agent performance

In [ ]:
# From: evaluators.py

# From: AI Agents Book, Chapter 18, Section 18.3
# File: evaluators.py
# Description: Simple evaluation functions for exact match and keyword checking


def evaluate_exact_match(response: str, expected: str) -> dict:
    """Check if response exactly matches expected output."""
    match = response.strip().lower() == expected.strip().lower()
    return {
        "metric": "exact_match",
        "score": 1.0 if match else 0.0,
        "passed": match
    }


def evaluate_contains_keywords(response: str, required_keywords: list[str]) -> dict:
    """Check if response contains all required keywords."""
    response_lower = response.lower()
    found = [kw for kw in required_keywords if kw.lower() in response_lower]
    missing = [kw for kw in required_keywords if kw.lower() not in response_lower]
    
    score = len(found) / len(required_keywords) if required_keywords else 1.0
    
    return {
        "metric": "keyword_coverage",
        "score": score,
        "found": found,
        "missing": missing,
        "passed": len(missing) == 0
    }


# Example usage
if __name__ == "__main__":
    # Test exact match
    result = evaluate_exact_match("Paris", "paris")
    print(f"Exact match: {result}")
    
    # Test keyword coverage
    response = "The capital of France is Paris, a beautiful city."
    result = evaluate_contains_keywords(response, ["Paris", "capital", "France"])
    print(f"Keyword coverage: {result}")


In [ ]:
# From: semantic_evaluator.py

# From: AI Agents Book, Chapter 18, Section 18.3
# File: semantic_evaluator.py
# Description: Evaluate responses using semantic similarity with embeddings

import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
import numpy as np

load_dotenv()


def cosine_similarity(vec1: list[float], vec2: list[float]) -> float:
    """Calculate cosine similarity between two vectors."""
    v1 = np.array(vec1)
    v2 = np.array(vec2)
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


class SemanticEvaluator:
    """Evaluate responses using semantic similarity."""
    
    def __init__(self):
        self.embeddings = OpenAIEmbeddings()
    
    def evaluate(self, response: str, reference: str, threshold: float = 0.85) -> dict:
        """Compare response to reference using embeddings."""
        response_embedding = self.embeddings.embed_query(response)
        reference_embedding = self.embeddings.embed_query(reference)
        
        similarity = cosine_similarity(response_embedding, reference_embedding)
        
        return {
            "metric": "semantic_similarity",
            "score": similarity,
            "threshold": threshold,
            "passed": similarity >= threshold
        }


# Example usage
if __name__ == "__main__":
    evaluator = SemanticEvaluator()
    
    # These have different words but same meaning
    response = "Paris is the capital of France."
    reference = "The capital city of France is Paris."
    
    result = evaluator.evaluate(response, reference)
    print(f"Semantic similarity: {result['score']:.3f}")
    print(f"Passed: {result['passed']}")


In [ ]:
# From: llm_judge.py

# From: AI Agents Book, Chapter 18, Section 18.3
# File: llm_judge.py
# Description: Use an LLM to evaluate agent responses (LLM-as-Judge pattern)

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

load_dotenv()


class LLMJudge:
    """Use an LLM to evaluate agent responses."""
    
    def __init__(self, model: str = "gpt-4o"):
        # Use a strong model for evaluation
        self.llm = ChatOpenAI(model=model, temperature=0)
    
    def evaluate(
        self, 
        question: str, 
        response: str, 
        criteria: str,
        reference: str | None = None
    ) -> dict:
        """
        Evaluate a response using LLM judgment.
        
        Args:
            question: The original question/prompt
            response: The agent's response to evaluate
            criteria: What to evaluate (e.g., "accuracy", "helpfulness")
            reference: Optional reference answer for comparison
        """
        system_prompt = """You are an expert evaluator. Your job is to evaluate 
AI assistant responses based on specific criteria.

Be objective and consistent. Provide a score from 1-5 where:
1 = Very poor, fails to meet the criteria
2 = Poor, partially meets criteria with significant issues  
3 = Acceptable, meets basic criteria but has room for improvement
4 = Good, meets criteria well with minor issues
5 = Excellent, fully meets or exceeds criteria

Always respond in this exact format:
SCORE: [1-5]
REASONING: [Your explanation]"""

        evaluation_prompt = f"""Evaluate the following response for {criteria}.

QUESTION: {question}

RESPONSE: {response}
"""
        if reference:
            evaluation_prompt += f"\nREFERENCE ANSWER: {reference}\n"
        
        evaluation_prompt += f"\nEvaluate for: {criteria}"
        
        result = self.llm.invoke([
            SystemMessage(content=system_prompt),
            HumanMessage(content=evaluation_prompt)
        ])
        
        # Parse the response
        content = result.content
        score = self._extract_score(content)
        reasoning = self._extract_reasoning(content)
        
        return {
            "metric": f"llm_judge_{criteria}",
            "score": score,
            "normalized_score": score / 5.0,  # Normalize to 0-1
            "reasoning": reasoning,
            "passed": score >= 3
        }
    
    def _extract_score(self, content: str) -> int:
        """Extract score from LLM response."""
        for line in content.split('\n'):
            if line.startswith('SCORE:'):
                try:
                    return int(line.split(':')[1].strip())
                except (ValueError, IndexError):
                    pass
        return 3  # Default to middle score if parsing fails
    
    def _extract_reasoning(self, content: str) -> str:
        """Extract reasoning from LLM response."""
        for i, line in enumerate(content.split('\n')):
            if line.startswith('REASONING:'):
                return '\n'.join(content.split('\n')[i:]).replace('REASONING:', '').strip()
        return content


# Example usage
if __name__ == "__main__":
    judge = LLMJudge()
    
    result = judge.evaluate(
        question="What is the capital of France?",
        response="Paris is the capital of France, located on the Seine River.",
        criteria="accuracy",
        reference="Paris"
    )
    
    print(f"Score: {result['score']}/5")
    print(f"Normalized: {result['normalized_score']:.2f}")
    print(f"Passed: {result['passed']}")
    print(f"Reasoning: {result['reasoning']}")


In [ ]:
# From: multi_criteria_evaluator.py

# From: AI Agents Book, Chapter 18, Section 18.3
# File: multi_criteria_evaluator.py
# Description: Evaluate responses across multiple criteria

from llm_judge import LLMJudge
from semantic_evaluator import SemanticEvaluator


class MultiCriteriaEvaluator:
    """Evaluate responses across multiple criteria."""
    
    def __init__(self):
        self.judge = LLMJudge()
        self.semantic = SemanticEvaluator()
    
    def evaluate(
        self,
        question: str,
        response: str,
        reference: str | None = None,
        criteria: list[str] | None = None
    ) -> dict:
        """Run comprehensive evaluation."""
        
        criteria = criteria or ["accuracy", "helpfulness", "clarity"]
        
        results = {
            "question": question,
            "response": response[:200] + "..." if len(response) > 200 else response,
            "evaluations": {},
            "summary": {}
        }
        
        # Run LLM judge for each criterion
        for criterion in criteria:
            eval_result = self.judge.evaluate(
                question=question,
                response=response,
                criteria=criterion,
                reference=reference
            )
            results["evaluations"][criterion] = eval_result
        
        # Add semantic similarity if reference provided
        if reference:
            semantic_result = self.semantic.evaluate(response, reference)
            results["evaluations"]["semantic_similarity"] = semantic_result
        
        # Calculate summary statistics
        scores = [
            e["normalized_score"] 
            for e in results["evaluations"].values() 
            if "normalized_score" in e
        ]
        
        results["summary"] = {
            "average_score": sum(scores) / len(scores) if scores else 0,
            "min_score": min(scores) if scores else 0,
            "max_score": max(scores) if scores else 0,
            "all_passed": all(
                e.get("passed", True) 
                for e in results["evaluations"].values()
            )
        }
        
        return results


# Example usage
if __name__ == "__main__":
    evaluator = MultiCriteriaEvaluator()
    
    result = evaluator.evaluate(
        question="How do I reset my password?",
        response="To reset your password, go to Settings > Security > Reset Password. You'll receive an email with a reset link.",
        reference="Navigate to Settings, then Security, and click Reset Password.",
        criteria=["accuracy", "helpfulness", "clarity"]
    )
    
    print("Evaluation Results:")
    print(f"Average Score: {result['summary']['average_score']:.2f}")
    print(f"All Passed: {result['summary']['all_passed']}")
    
    for criterion, eval_data in result["evaluations"].items():
        print(f"  {criterion}: {eval_data.get('normalized_score', eval_data.get('score', 'N/A'))}")


In [ ]:
# From: tool_usage_evaluator.py

# From: AI Agents Book, Chapter 18, Section 18.3
# File: tool_usage_evaluator.py
# Description: Evaluate how well an agent uses its tools


class ToolUsageEvaluator:
    """Evaluate how well an agent uses its tools."""
    
    def evaluate_tool_selection(
        self,
        query: str,
        tools_used: list[str],
        expected_tools: list[str]
    ) -> dict:
        """Evaluate if the agent selected appropriate tools."""
        used_set = set(tools_used)
        expected_set = set(expected_tools)
        
        correct = used_set & expected_set  # Tools correctly used
        missed = expected_set - used_set    # Tools that should have been used
        extra = used_set - expected_set     # Tools used unnecessarily
        
        # Precision: of tools used, how many were correct?
        precision = len(correct) / len(used_set) if used_set else 1.0
        
        # Recall: of tools needed, how many were used?
        recall = len(correct) / len(expected_set) if expected_set else 1.0
        
        # F1 score balances precision and recall
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        return {
            "metric": "tool_selection",
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "correct_tools": list(correct),
            "missed_tools": list(missed),
            "extra_tools": list(extra),
            "passed": recall >= 0.8 and precision >= 0.5
        }
    
    def evaluate_tool_arguments(
        self,
        tool_calls: list[dict],
        expected_calls: list[dict]
    ) -> dict:
        """Evaluate if tools were called with correct arguments."""
        if not expected_calls:
            return {"metric": "tool_arguments", "score": 1.0, "passed": True}
        
        correct_count = 0
        details = []
        
        for expected in expected_calls:
            # Find matching tool call
            matching = [
                tc for tc in tool_calls 
                if tc.get("tool") == expected.get("tool")
            ]
            
            if not matching:
                details.append({
                    "tool": expected["tool"],
                    "status": "not_called",
                    "correct": False
                })
                continue
            
            # Check arguments
            actual = matching[0]
            args_match = self._compare_arguments(
                actual.get("arguments", {}),
                expected.get("arguments", {})
            )
            
            if args_match:
                correct_count += 1
            
            details.append({
                "tool": expected["tool"],
                "status": "correct" if args_match else "wrong_arguments",
                "expected_args": expected.get("arguments"),
                "actual_args": actual.get("arguments"),
                "correct": args_match
            })
        
        score = correct_count / len(expected_calls)
        
        return {
            "metric": "tool_arguments",
            "score": score,
            "details": details,
            "passed": score >= 0.8
        }
    
    def _compare_arguments(self, actual: dict, expected: dict) -> bool:
        """Compare tool arguments, allowing for minor variations."""
        for key, expected_value in expected.items():
            if key not in actual:
                return False
            
            actual_value = actual[key]
            
            # String comparison (case-insensitive, stripped)
            if isinstance(expected_value, str) and isinstance(actual_value, str):
                if expected_value.strip().lower() != actual_value.strip().lower():
                    return False
            # Numeric comparison (allow small differences)
            elif isinstance(expected_value, (int, float)) and isinstance(actual_value, (int, float)):
                if abs(expected_value - actual_value) > 0.01:
                    return False
            # Exact comparison for other types
            elif expected_value != actual_value:
                return False
        
        return True


# Example usage
if __name__ == "__main__":
    evaluator = ToolUsageEvaluator()
    
    # Test tool selection
    result = evaluator.evaluate_tool_selection(
        query="What's the weather in Paris?",
        tools_used=["weather_api", "location_lookup"],
        expected_tools=["weather_api"]
    )
    print(f"Tool Selection: F1={result['f1_score']:.2f}, Passed={result['passed']}")
    
    # Test tool arguments
    result = evaluator.evaluate_tool_arguments(
        tool_calls=[
            {"tool": "weather_api", "arguments": {"city": "Paris", "units": "celsius"}}
        ],
        expected_calls=[
            {"tool": "weather_api", "arguments": {"city": "Paris"}}
        ]
    )
    print(f"Tool Arguments: Score={result['score']:.2f}, Passed={result['passed']}")


In [ ]:
# From: evaluation_pipeline.py

# From: AI Agents Book, Chapter 18, Section 18.3
# File: evaluation_pipeline.py
# Description: Run systematic evaluations across test cases

from dataclasses import dataclass
from datetime import datetime
import json

from multi_criteria_evaluator import MultiCriteriaEvaluator
from tool_usage_evaluator import ToolUsageEvaluator


@dataclass
class EvaluationCase:
    """A single evaluation test case."""
    id: str
    question: str
    expected_response: str | None = None
    expected_tools: list[str] | None = None
    criteria: list[str] | None = None
    metadata: dict | None = None


class EvaluationPipeline:
    """Run systematic evaluations across test cases."""
    
    def __init__(self, agent_fn):
        """
        Initialize with a function that takes a question and returns a response.
        
        agent_fn should return a dict with at least:
        - "response": str
        - "tools_used": list[str] (optional)
        """
        self.agent_fn = agent_fn
        self.multi_evaluator = MultiCriteriaEvaluator()
        self.tool_evaluator = ToolUsageEvaluator()
    
    def run(self, test_cases: list[EvaluationCase]) -> dict:
        """Run evaluation on all test cases."""
        results = {
            "timestamp": datetime.now().isoformat(),
            "total_cases": len(test_cases),
            "cases": [],
            "aggregate": {}
        }
        
        all_scores = []
        passed_count = 0
        
        for case in test_cases:
            case_result = self._evaluate_case(case)
            results["cases"].append(case_result)
            
            if case_result["summary"]["all_passed"]:
                passed_count += 1
            
            all_scores.append(case_result["summary"]["average_score"])
        
        # Aggregate statistics
        results["aggregate"] = {
            "pass_rate": passed_count / len(test_cases) if test_cases else 0,
            "average_score": sum(all_scores) / len(all_scores) if all_scores else 0,
            "passed_count": passed_count,
            "failed_count": len(test_cases) - passed_count
        }
        
        return results
    
    def _evaluate_case(self, case: EvaluationCase) -> dict:
        """Evaluate a single test case."""
        # Get agent response
        agent_result = self.agent_fn(case.question)
        response = agent_result.get("response", "")
        tools_used = agent_result.get("tools_used", [])
        
        # Run multi-criteria evaluation
        eval_result = self.multi_evaluator.evaluate(
            question=case.question,
            response=response,
            reference=case.expected_response,
            criteria=case.criteria
        )
        
        # Add tool evaluation if expected tools specified
        if case.expected_tools:
            tool_result = self.tool_evaluator.evaluate_tool_selection(
                query=case.question,
                tools_used=tools_used,
                expected_tools=case.expected_tools
            )
            eval_result["evaluations"]["tool_selection"] = tool_result
        
        eval_result["case_id"] = case.id
        return eval_result


# Example usage
if __name__ == "__main__":
    # Mock agent function for demonstration
    def mock_agent(question: str) -> dict:
        return {
            "response": f"This is a mock response to: {question}",
            "tools_used": ["search", "calculator"]
        }
    
    # Create test cases
    test_cases = [
        EvaluationCase(
            id="test_001",
            question="What is 2 + 2?",
            expected_response="4",
            expected_tools=["calculator"],
            criteria=["accuracy", "clarity"]
        ),
        EvaluationCase(
            id="test_002",
            question="What is the capital of France?",
            expected_response="Paris",
            criteria=["accuracy", "helpfulness"]
        )
    ]
    
    # Run pipeline
    pipeline = EvaluationPipeline(mock_agent)
    results = pipeline.run(test_cases)
    
    print(f"Total Cases: {results['total_cases']}")
    print(f"Pass Rate: {results['aggregate']['pass_rate']:.1%}")
    print(f"Average Score: {results['aggregate']['average_score']:.2f}")


In [ ]:
# From: evaluation_tracker.py

# From: AI Agents Book, Chapter 18, Section 18.3
# File: evaluation_tracker.py
# Description: Track evaluation results over time

import json
from datetime import datetime


class EvaluationTracker:
    """Track evaluation results over time."""
    
    def __init__(self, storage_path: str = "evaluations.json"):
        self.storage_path = storage_path
        self.history = self._load_history()
    
    def _load_history(self) -> list:
        """Load evaluation history from storage."""
        try:
            with open(self.storage_path, 'r') as f:
                return json.load(f)
        except FileNotFoundError:
            return []
    
    def _save_history(self):
        """Save evaluation history to storage."""
        with open(self.storage_path, 'w') as f:
            json.dump(self.history, f, indent=2)
    
    def record(self, evaluation_result: dict, version: str = "unknown"):
        """Record an evaluation result."""
        record = {
            "timestamp": datetime.now().isoformat(),
            "version": version,
            "aggregate": evaluation_result.get("aggregate", {}),
            "total_cases": evaluation_result.get("total_cases", 0)
        }
        self.history.append(record)
        self._save_history()
    
    def get_trend(self, metric: str = "average_score", last_n: int = 10) -> list:
        """Get trend for a specific metric."""
        recent = self.history[-last_n:]
        return [
            {
                "timestamp": r["timestamp"],
                "version": r["version"],
                "value": r["aggregate"].get(metric, 0)
            }
            for r in recent
        ]
    
    def compare_versions(self, version_a: str, version_b: str) -> dict:
        """Compare metrics between two versions."""
        results_a = [r for r in self.history if r["version"] == version_a]
        results_b = [r for r in self.history if r["version"] == version_b]
        
        if not results_a or not results_b:
            return {"error": "One or both versions not found"}
        
        # Use most recent result for each version
        latest_a = results_a[-1]["aggregate"]
        latest_b = results_b[-1]["aggregate"]
        
        comparison = {}
        for metric in latest_a.keys():
            if metric in latest_b:
                val_a = latest_a[metric]
                val_b = latest_b[metric]
                if isinstance(val_a, (int, float)) and isinstance(val_b, (int, float)):
                    comparison[metric] = {
                        "version_a": val_a,
                        "version_b": val_b,
                        "difference": val_b - val_a,
                        "percent_change": ((val_b - val_a) / val_a * 100) if val_a != 0 else 0
                    }
        
        return comparison


# Example usage
if __name__ == "__main__":
    tracker = EvaluationTracker("test_evaluations.json")
    
    # Record some sample evaluations
    tracker.record({
        "total_cases": 10,
        "aggregate": {"pass_rate": 0.7, "average_score": 0.65}
    }, version="v1.0")
    
    tracker.record({
        "total_cases": 10,
        "aggregate": {"pass_rate": 0.8, "average_score": 0.75}
    }, version="v1.1")
    
    # Get trend
    trend = tracker.get_trend("average_score")
    print("Score Trend:")
    for point in trend:
        print(f"  {point['version']}: {point['value']:.2f}")
    
    # Compare versions
    comparison = tracker.compare_versions("v1.0", "v1.1")
    print("\nVersion Comparison (v1.0 -> v1.1):")
    for metric, data in comparison.items():
        print(f"  {metric}: {data['percent_change']:+.1f}%")


---
### Section 18.3 Exercises

### Exercise 18.3.1: Custom Evaluation Rubric

Create an evaluation rubric for a customer service agent that rates responses on four criteria: accuracy (factual correctness), empathy (emotional tone), actionability (does it help solve the problem?), and policy compliance (stays within company guidelines). Build an LLMJudge-based evaluator that scores each criterion and returns detailed feedback.

In [ ]:
# Your code here


### Exercise 18.3.2: Comparative Evaluation

Build an evaluator that compares two different agent responses to the same question and decides which is better. It should explain its reasoning and output a clear winner (or tie). This is useful for A/B testing different agent versions.

In [ ]:
# Your code here


### Exercise 18.3.3: Evaluation Dashboard

Create a simple evaluation reporting function that takes a list of evaluation results and generates a summary report. The report should include: overall pass rate, breakdown by criterion, the worst-performing cases (for debugging), and trend information if historical data is available.

In [ ]:
# Your code here


---
## Section 18.4: Creating test datasets

In [ ]:
# From: dataset_generators.py

# From: AI Agents Book, Chapter 18, Section 18.4
# File: dataset_generators.py
# Description: Tools for generating synthetic test datasets

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()


def generate_variations(original_query: str, llm, count: int = 5) -> list[str]:
    """
    Generate phrasing variations of a query.
    
    Takes a single query and generates multiple phrasings that mean the same thing.
    Useful for testing that your agent handles different phrasings consistently.
    
    Args:
        original_query: The original user query
        llm: A language model to generate variations
        count: Number of variations to generate
    
    Returns:
        List of query variations
    """
    prompt = f"""Generate {count} different ways a user might ask this same question.
Vary the formality, length, and phrasing while keeping the meaning identical.

Original: {original_query}

Variations (one per line, numbered):"""
    
    response = llm.invoke(prompt)
    
    # Parse the numbered list from response
    variations = []
    for line in response.content.split('\n'):
        line = line.strip()
        if line and len(line) > 2:
            # Remove numbering like "1.", "1)", "1:"
            if line[0].isdigit():
                # Find where the actual text starts
                for i, char in enumerate(line):
                    if char in '.):' and i < 3:
                        line = line[i+1:].strip()
                        break
            if line:
                variations.append(line)
    
    return variations[:count]


def generate_scenario_cases(scenario_description: str, count: int, llm) -> list[dict]:
    """
    Generate test cases for a specific scenario.
    
    Args:
        scenario_description: Description of the scenario to generate cases for
        count: Number of test cases to generate
        llm: A language model to generate cases
    
    Returns:
        List of test case dictionaries with 'query' and 'should_address' keys
    """
    prompt = f"""Generate {count} realistic user queries for this scenario:
{scenario_description}

For each query, also provide what a correct response should address.

Format each as:
QUERY: [the user's question]
SHOULD_ADDRESS: [key points the response must cover]

Generate {count} different cases:"""
    
    response = llm.invoke(prompt)
    
    # Parse into structured cases
    cases = []
    current_query = None
    current_should_address = None
    
    for line in response.content.split('\n'):
        line = line.strip()
        if line.startswith('QUERY:'):
            if current_query and current_should_address:
                cases.append({
                    'query': current_query,
                    'should_address': current_should_address
                })
            current_query = line[6:].strip()
            current_should_address = None
        elif line.startswith('SHOULD_ADDRESS:'):
            current_should_address = line[15:].strip()
    
    # Don't forget the last case
    if current_query and current_should_address:
        cases.append({
            'query': current_query,
            'should_address': current_should_address
        })
    
    return cases[:count]


def generate_edge_cases(llm, edge_type: str = "ambiguous") -> list[str]:
    """
    Generate edge case queries for testing robustness.
    
    Args:
        llm: A language model to generate cases
        edge_type: Type of edge case to generate
    
    Returns:
        List of edge case queries
    """
    edge_case_prompts = {
        "ambiguous": "Generate 5 queries where the user's intent is ambiguous and could be interpreted multiple ways",
        "multi_request": "Generate 5 queries that combine multiple unrelated requests in a single message",
        "typos": "Generate 5 queries with realistic typos and grammatical errors",
        "out_of_scope": "Generate 5 queries that are just barely outside a typical customer service agent's scope",
        "boundary": "Generate 5 queries that test the boundaries between different categories (e.g., is this a billing question or a technical question?)",
    }
    
    prompt = edge_case_prompts.get(edge_type, edge_case_prompts["ambiguous"])
    prompt += "\n\nList each query on its own line:"
    
    response = llm.invoke(prompt)
    
    # Parse into list
    cases = []
    for line in response.content.split('\n'):
        line = line.strip()
        if line and len(line) > 10:  # Filter out empty/short lines
            # Remove numbering
            if line[0].isdigit() and len(line) > 3:
                for i, char in enumerate(line):
                    if char in '.):' and i < 3:
                        line = line[i+1:].strip()
                        break
            if line:
                cases.append(line)
    
    return cases[:5]


def generate_adversarial_cases(llm) -> dict[str, list[str]]:
    """
    Generate a suite of adversarial test cases.
    
    Returns:
        Dictionary mapping adversarial category to list of test queries
    """
    categories = {
        "ambiguity": "Generate queries where the correct answer is 'I need more information' because key details are missing",
        "boundary": "Generate queries at the edge of a customer service agent's capabilities - almost but not quite in scope",
        "consistency": "Generate 5 different phrasings of the same factual question to test for consistent answers",
    }
    
    results = {}
    for category, prompt in categories.items():
        full_prompt = f"{prompt}\n\nList 5 examples, one per line:"
        response = llm.invoke(full_prompt)
        
        cases = []
        for line in response.content.split('\n'):
            line = line.strip()
            if line and len(line) > 10:
                if line[0].isdigit():
                    for i, char in enumerate(line):
                        if char in '.):' and i < 3:
                            line = line[i+1:].strip()
                            break
                if line:
                    cases.append(line)
        
        results[category] = cases[:5]
    
    return results


# Example usage
if __name__ == "__main__":
    from langchain_openai import ChatOpenAI
    
    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
    
    # Generate variations
    print("=== Query Variations ===")
    original = "How do I reset my password?"
    variations = generate_variations(original, llm)
    print(f"Original: {original}")
    for i, var in enumerate(variations, 1):
        print(f"  {i}. {var}")
    
    # Generate scenario cases
    print("\n=== Scenario Cases ===")
    scenario = "A user is frustrated because their order hasn't arrived after 2 weeks"
    cases = generate_scenario_cases(scenario, 3, llm)
    for case in cases:
        print(f"Query: {case['query']}")
        print(f"Should address: {case['should_address']}\n")
    
    # Generate edge cases
    print("=== Edge Cases (Ambiguous) ===")
    edge_cases = generate_edge_cases(llm, "ambiguous")
    for case in edge_cases:
        print(f"  - {case}")


In [ ]:
# From: dataset_manager.py

# From: AI Agents Book, Chapter 18, Section 18.4
# File: dataset_manager.py
# Description: Manage test datasets - load, validate, filter, and analyze

import json
from pathlib import Path
from datetime import datetime
from collections import Counter


class TestDataset:
    """Manage a test dataset for agent evaluation."""
    
    def __init__(self, path: str | None = None):
        """
        Initialize dataset, optionally loading from a file.
        
        Args:
            path: Path to JSON dataset file
        """
        self.cases = []
        self.metadata = {
            "dataset_version": "1.0",
            "created": datetime.now().isoformat(),
            "description": ""
        }
        
        if path:
            self.load(path)
    
    def load(self, path: str) -> None:
        """Load dataset from JSON file."""
        with open(path, 'r') as f:
            data = json.load(f)
        
        self.metadata = {k: v for k, v in data.items() if k != 'cases'}
        self.cases = data.get('cases', [])
        
        print(f"Loaded {len(self.cases)} cases from {path}")
    
    def save(self, path: str) -> None:
        """Save dataset to JSON file."""
        data = {**self.metadata, "cases": self.cases}
        
        with open(path, 'w') as f:
            json.dump(data, f, indent=2)
        
        print(f"Saved {len(self.cases)} cases to {path}")
    
    def add_case(self, case: dict) -> None:
        """Add a single test case."""
        # Ensure required fields
        required = ['id', 'query']
        for field in required:
            if field not in case:
                raise ValueError(f"Case missing required field: {field}")
        
        # Check for duplicate IDs
        existing_ids = {c['id'] for c in self.cases}
        if case['id'] in existing_ids:
            raise ValueError(f"Duplicate case ID: {case['id']}")
        
        self.cases.append(case)
    
    def get_case(self, case_id: str) -> dict | None:
        """Get a case by ID."""
        for case in self.cases:
            if case['id'] == case_id:
                return case
        return None
    
    def filter(
        self,
        category: str | None = None,
        difficulty: str | None = None,
        source: str | None = None,
        tags: list[str] | None = None
    ) -> list[dict]:
        """
        Filter cases by criteria.
        
        Args:
            category: Filter by category
            difficulty: Filter by difficulty (easy, medium, hard)
            source: Filter by source (golden, real_user, synthetic, adversarial)
            tags: Filter by tags (case must have ALL specified tags)
        
        Returns:
            List of matching cases
        """
        results = self.cases
        
        if category:
            results = [c for c in results if c.get('category') == category]
        
        if difficulty:
            results = [c for c in results if c.get('difficulty') == difficulty]
        
        if source:
            results = [c for c in results if c.get('source') == source]
        
        if tags:
            results = [c for c in results 
                      if all(t in c.get('tags', []) for t in tags)]
        
        return results
    
    def get_statistics(self) -> dict:
        """Get dataset statistics."""
        stats = {
            "total_cases": len(self.cases),
            "by_category": Counter(c.get('category', 'unknown') for c in self.cases),
            "by_difficulty": Counter(c.get('difficulty', 'unknown') for c in self.cases),
            "by_source": Counter(c.get('source', 'unknown') for c in self.cases),
            "all_tags": Counter(tag for c in self.cases for tag in c.get('tags', []))
        }
        return stats
    
    def print_summary(self) -> None:
        """Print a summary of the dataset."""
        stats = self.get_statistics()
        
        print(f"\n{'='*50}")
        print(f"Dataset Summary")
        print(f"{'='*50}")
        print(f"Version: {self.metadata.get('dataset_version', 'unknown')}")
        print(f"Total Cases: {stats['total_cases']}")
        
        print(f"\nBy Category:")
        for cat, count in stats['by_category'].most_common():
            print(f"  {cat}: {count}")
        
        print(f"\nBy Difficulty:")
        for diff, count in stats['by_difficulty'].most_common():
            print(f"  {diff}: {count}")
        
        print(f"\nBy Source:")
        for src, count in stats['by_source'].most_common():
            print(f"  {src}: {count}")
        
        print(f"\nTop Tags:")
        for tag, count in stats['all_tags'].most_common(5):
            print(f"  {tag}: {count}")
    
    def validate(self) -> list[str]:
        """
        Validate dataset integrity.
        
        Returns:
            List of validation errors (empty if valid)
        """
        errors = []
        ids_seen = set()
        
        for i, case in enumerate(self.cases):
            # Check required fields
            if 'id' not in case:
                errors.append(f"Case {i}: missing 'id' field")
            elif case['id'] in ids_seen:
                errors.append(f"Case {i}: duplicate id '{case['id']}'")
            else:
                ids_seen.add(case['id'])
            
            if 'query' not in case:
                errors.append(f"Case {case.get('id', i)}: missing 'query' field")
            
            # Check expected_response structure
            if 'expected_response' in case:
                exp = case['expected_response']
                if not isinstance(exp, dict):
                    errors.append(f"Case {case.get('id', i)}: 'expected_response' should be a dict")
        
        if errors:
            print(f"Validation found {len(errors)} errors")
        else:
            print("Dataset validation passed!")
        
        return errors
    
    def to_evaluation_cases(self) -> list[dict]:
        """
        Convert to format suitable for evaluation pipeline.
        
        Returns:
            List of cases formatted for EvaluationPipeline
        """
        eval_cases = []
        for case in self.cases:
            eval_case = {
                "id": case["id"],
                "question": case["query"],
                "expected_response": case.get("expected_response", {}).get("reference_answer"),
                "criteria": self._infer_criteria(case),
                "metadata": {
                    "category": case.get("category"),
                    "difficulty": case.get("difficulty"),
                    "source": case.get("source"),
                    "tags": case.get("tags", [])
                }
            }
            eval_cases.append(eval_case)
        return eval_cases
    
    def _infer_criteria(self, case: dict) -> list[str]:
        """Infer evaluation criteria from case properties."""
        criteria = ["accuracy", "helpfulness"]
        
        tags = case.get("tags", [])
        if "complaint" in tags:
            criteria.append("empathy")
        if "needs_clarification" in tags or "ambiguous" in tags:
            criteria.append("appropriateness")
        
        return criteria


# Example usage
if __name__ == "__main__":
    # Load the sample dataset
    dataset = TestDataset("sample_dataset.json")
    
    # Print summary
    dataset.print_summary()
    
    # Validate
    errors = dataset.validate()
    if errors:
        for error in errors:
            print(f"  - {error}")
    
    # Filter examples
    print("\n=== Adversarial Cases ===")
    adversarial = dataset.filter(source="adversarial")
    for case in adversarial:
        print(f"  {case['id']}: {case['query'][:50]}...")
    
    print("\n=== Easy Cases ===")
    easy = dataset.filter(difficulty="easy")
    for case in easy:
        print(f"  {case['id']}: {case['query'][:50]}...")
    
    # Convert to evaluation format
    print("\n=== First 3 as Evaluation Cases ===")
    eval_cases = dataset.to_evaluation_cases()[:3]
    for ec in eval_cases:
        print(f"  {ec['id']}: criteria={ec['criteria']}")


---
## Section 18.5: Measuring accuracy and reliability

In [ ]:
# From: accuracy_calculator.py

# From: AI Agents Book, Chapter 18, Section 18.5
# File: accuracy_calculator.py
# Description: Calculate accuracy metrics with severity weighting and category breakdown

from dataclasses import dataclass
from collections import defaultdict
import math


@dataclass
class EvaluationResult:
    """Result of a single evaluation."""
    case_id: str
    passed: bool
    category: str
    severity: str = "minor"  # critical, major, minor, trivial
    score: float = 0.0  # 0.0 to 1.0


class AccuracyCalculator:
    """
    Calculate accuracy metrics with severity weighting and category breakdown.
    
    Implements concepts from Section 18.5:
    - Weighted accuracy by error severity
    - Category-specific breakdowns
    - Confidence intervals
    """
    
    # Default severity weights (higher = more impactful)
    DEFAULT_WEIGHTS = {
        "critical": 10.0,
        "major": 5.0,
        "minor": 2.0,
        "trivial": 1.0
    }
    
    def __init__(self, severity_weights: dict | None = None):
        """
        Initialize calculator.
        
        Args:
            severity_weights: Custom weights for severity levels
        """
        self.weights = severity_weights or self.DEFAULT_WEIGHTS
        self.results: list[EvaluationResult] = []
    
    def add_result(self, result: EvaluationResult) -> None:
        """Add an evaluation result."""
        self.results.append(result)
    
    def add_results(self, results: list[EvaluationResult]) -> None:
        """Add multiple evaluation results."""
        self.results.extend(results)
    
    def overall_accuracy(self) -> float:
        """Calculate simple overall accuracy (pass rate)."""
        if not self.results:
            return 0.0
        passed = sum(1 for r in self.results if r.passed)
        return passed / len(self.results)
    
    def weighted_accuracy(self) -> float:
        """
        Calculate weighted accuracy considering error severity.
        
        Errors with higher severity have more impact on the score.
        """
        if not self.results:
            return 0.0
        
        total_weight = 0.0
        weighted_pass = 0.0
        
        for r in self.results:
            weight = self.weights.get(r.severity, 1.0)
            total_weight += weight
            if r.passed:
                weighted_pass += weight
        
        return weighted_pass / total_weight if total_weight > 0 else 0.0
    
    def accuracy_by_category(self) -> dict[str, dict]:
        """
        Break down accuracy by category.
        
        Returns dict with category -> {accuracy, count, passed, failed}
        """
        categories = defaultdict(lambda: {"passed": 0, "failed": 0})
        
        for r in self.results:
            if r.passed:
                categories[r.category]["passed"] += 1
            else:
                categories[r.category]["failed"] += 1
        
        breakdown = {}
        for cat, counts in categories.items():
            total = counts["passed"] + counts["failed"]
            breakdown[cat] = {
                "accuracy": counts["passed"] / total if total > 0 else 0.0,
                "count": total,
                "passed": counts["passed"],
                "failed": counts["failed"]
            }
        
        return breakdown
    
    def accuracy_by_severity(self) -> dict[str, dict]:
        """Break down accuracy by error severity level."""
        severities = defaultdict(lambda: {"passed": 0, "failed": 0})
        
        for r in self.results:
            if r.passed:
                severities[r.severity]["passed"] += 1
            else:
                severities[r.severity]["failed"] += 1
        
        breakdown = {}
        for sev, counts in severities.items():
            total = counts["passed"] + counts["failed"]
            breakdown[sev] = {
                "accuracy": counts["passed"] / total if total > 0 else 0.0,
                "count": total,
                "error_rate": counts["failed"] / total if total > 0 else 0.0
            }
        
        return breakdown
    
    def critical_error_rate(self) -> float:
        """Calculate the rate of critical errors specifically."""
        critical_results = [r for r in self.results if r.severity == "critical"]
        if not critical_results:
            return 0.0
        failed = sum(1 for r in critical_results if not r.passed)
        return failed / len(critical_results)
    
    def confidence_interval(self, confidence: float = 0.95) -> tuple[float, float]:
        """
        Calculate confidence interval for overall accuracy.
        
        Uses normal approximation to binomial.
        
        Args:
            confidence: Confidence level (default 0.95 for 95% CI)
        
        Returns:
            Tuple of (lower_bound, upper_bound)
        """
        n = len(self.results)
        if n == 0:
            return (0.0, 0.0)
        
        p = self.overall_accuracy()
        
        # Z-score for confidence level
        z_scores = {0.90: 1.645, 0.95: 1.96, 0.99: 2.576}
        z = z_scores.get(confidence, 1.96)
        
        # Standard error
        se = math.sqrt(p * (1 - p) / n)
        
        # Confidence interval
        margin = z * se
        lower = max(0.0, p - margin)
        upper = min(1.0, p + margin)
        
        return (lower, upper)
    
    def summary(self) -> dict:
        """Generate a complete summary of accuracy metrics."""
        ci_lower, ci_upper = self.confidence_interval()
        
        return {
            "total_cases": len(self.results),
            "overall_accuracy": self.overall_accuracy(),
            "weighted_accuracy": self.weighted_accuracy(),
            "confidence_interval_95": {
                "lower": ci_lower,
                "upper": ci_upper
            },
            "critical_error_rate": self.critical_error_rate(),
            "by_category": self.accuracy_by_category(),
            "by_severity": self.accuracy_by_severity()
        }
    
    def print_report(self) -> None:
        """Print a formatted accuracy report."""
        summary = self.summary()
        
        print("\n" + "=" * 60)
        print("ACCURACY REPORT")
        print("=" * 60)
        
        print(f"\nTotal Cases: {summary['total_cases']}")
        print(f"Overall Accuracy: {summary['overall_accuracy']:.1%}")
        print(f"Weighted Accuracy: {summary['weighted_accuracy']:.1%}")
        
        ci = summary['confidence_interval_95']
        print(f"95% Confidence Interval: [{ci['lower']:.1%}, {ci['upper']:.1%}]")
        
        print(f"\nCritical Error Rate: {summary['critical_error_rate']:.1%}")
        
        print("\n--- By Category ---")
        for cat, stats in summary['by_category'].items():
            print(f"  {cat}: {stats['accuracy']:.1%} "
                  f"({stats['passed']}/{stats['count']})")
        
        print("\n--- By Severity ---")
        for sev in ["critical", "major", "minor", "trivial"]:
            if sev in summary['by_severity']:
                stats = summary['by_severity'][sev]
                print(f"  {sev}: {stats['accuracy']:.1%} accuracy, "
                      f"{stats['error_rate']:.1%} error rate "
                      f"(n={stats['count']})")
        
        print("=" * 60)


# Example usage
if __name__ == "__main__":
    calc = AccuracyCalculator()
    
    # Simulate evaluation results
    results = [
        # Password reset - mostly passes
        EvaluationResult("pr_001", True, "password_reset", "minor"),
        EvaluationResult("pr_002", True, "password_reset", "minor"),
        EvaluationResult("pr_003", True, "password_reset", "minor"),
        EvaluationResult("pr_004", False, "password_reset", "minor"),
        
        # Order status - good performance
        EvaluationResult("os_001", True, "order_status", "major"),
        EvaluationResult("os_002", True, "order_status", "major"),
        EvaluationResult("os_003", True, "order_status", "major"),
        EvaluationResult("os_004", True, "order_status", "major"),
        EvaluationResult("os_005", False, "order_status", "major"),
        
        # Refunds - problematic area
        EvaluationResult("rf_001", True, "refunds", "critical"),
        EvaluationResult("rf_002", False, "refunds", "critical"),
        EvaluationResult("rf_003", False, "refunds", "critical"),
        EvaluationResult("rf_004", True, "refunds", "critical"),
        
        # Technical - decent
        EvaluationResult("tech_001", True, "technical", "major"),
        EvaluationResult("tech_002", True, "technical", "major"),
        EvaluationResult("tech_003", False, "technical", "major"),
    ]
    
    calc.add_results(results)
    calc.print_report()


In [ ]:
# From: reliability_metrics.py

# From: AI Agents Book, Chapter 18, Section 18.5
# File: reliability_metrics.py
# Description: Measure reliability through consistency, multiple runs, and statistical comparison

from dataclasses import dataclass
import math
from collections import defaultdict


@dataclass 
class RunResult:
    """Result of a single run of a test case."""
    case_id: str
    run_number: int
    passed: bool
    score: float = 0.0


class ReliabilityAnalyzer:
    """
    Analyze reliability through consistency and multiple runs.
    
    Implements concepts from Section 18.5:
    - Multiple runs per test case
    - Consistency measurement
    - Statistical significance testing
    """
    
    def __init__(self):
        self.runs: dict[str, list[RunResult]] = defaultdict(list)
    
    def add_run(self, result: RunResult) -> None:
        """Add a run result for a test case."""
        self.runs[result.case_id].append(result)
    
    def add_runs(self, results: list[RunResult]) -> None:
        """Add multiple run results."""
        for result in results:
            self.add_run(result)
    
    def consistency_score(self, case_id: str) -> float:
        """
        Calculate consistency for a specific test case.
        
        Returns 1.0 if all runs have same outcome, 0.0 if split 50/50.
        """
        runs = self.runs.get(case_id, [])
        if len(runs) < 2:
            return 1.0  # Can't measure consistency with < 2 runs
        
        pass_count = sum(1 for r in runs if r.passed)
        pass_rate = pass_count / len(runs)
        
        # Consistency is distance from 50/50
        return abs(pass_rate - 0.5) * 2
    
    def overall_consistency(self) -> float:
        """Calculate average consistency across all test cases."""
        if not self.runs:
            return 0.0
        
        consistencies = [self.consistency_score(case_id) 
                        for case_id in self.runs.keys()]
        return sum(consistencies) / len(consistencies)
    
    def pass_rate_per_case(self) -> dict[str, float]:
        """Calculate pass rate for each test case across runs."""
        rates = {}
        for case_id, runs in self.runs.items():
            if runs:
                rates[case_id] = sum(1 for r in runs if r.passed) / len(runs)
        return rates
    
    def aggregate_pass_rate(self) -> float:
        """Calculate overall pass rate across all runs."""
        all_runs = [r for runs in self.runs.values() for r in runs]
        if not all_runs:
            return 0.0
        return sum(1 for r in all_runs if r.passed) / len(all_runs)
    
    def worst_case_accuracy(self) -> float:
        """
        Calculate accuracy using worst run for each case.
        
        Conservative estimate - if any run fails, case counts as failed.
        """
        if not self.runs:
            return 0.0
        
        worst_pass = 0
        for case_id, runs in self.runs.items():
            # Case passes only if ALL runs pass
            if all(r.passed for r in runs):
                worst_pass += 1
        
        return worst_pass / len(self.runs)
    
    def best_case_accuracy(self) -> float:
        """
        Calculate accuracy using best run for each case.
        
        Optimistic estimate - if any run passes, case counts as passed.
        """
        if not self.runs:
            return 0.0
        
        best_pass = 0
        for case_id, runs in self.runs.items():
            # Case passes if ANY run passes
            if any(r.passed for r in runs):
                best_pass += 1
        
        return best_pass / len(self.runs)
    
    def flaky_cases(self, threshold: float = 0.8) -> list[str]:
        """
        Identify flaky test cases (inconsistent results).
        
        Args:
            threshold: Consistency below this is considered flaky
        
        Returns:
            List of case IDs that are flaky
        """
        flaky = []
        for case_id in self.runs.keys():
            if self.consistency_score(case_id) < threshold:
                flaky.append(case_id)
        return flaky
    
    def summary(self) -> dict:
        """Generate reliability summary."""
        return {
            "total_cases": len(self.runs),
            "total_runs": sum(len(runs) for runs in self.runs.values()),
            "aggregate_pass_rate": self.aggregate_pass_rate(),
            "worst_case_accuracy": self.worst_case_accuracy(),
            "best_case_accuracy": self.best_case_accuracy(),
            "overall_consistency": self.overall_consistency(),
            "flaky_cases": self.flaky_cases()
        }
    
    def print_report(self) -> None:
        """Print a formatted reliability report."""
        summary = self.summary()
        
        print("\n" + "=" * 60)
        print("RELIABILITY REPORT")
        print("=" * 60)
        
        print(f"\nTest Cases: {summary['total_cases']}")
        print(f"Total Runs: {summary['total_runs']}")
        avg_runs = summary['total_runs'] / summary['total_cases'] if summary['total_cases'] > 0 else 0
        print(f"Average Runs per Case: {avg_runs:.1f}")
        
        print(f"\nAggregate Pass Rate: {summary['aggregate_pass_rate']:.1%}")
        print(f"Worst-Case Accuracy: {summary['worst_case_accuracy']:.1%}")
        print(f"Best-Case Accuracy: {summary['best_case_accuracy']:.1%}")
        print(f"Overall Consistency: {summary['overall_consistency']:.1%}")
        
        if summary['flaky_cases']:
            print(f"\n⚠️  Flaky Cases ({len(summary['flaky_cases'])}):")
            for case_id in summary['flaky_cases'][:5]:  # Show first 5
                rates = self.pass_rate_per_case()
                print(f"    {case_id}: {rates[case_id]:.0%} pass rate")
            if len(summary['flaky_cases']) > 5:
                print(f"    ... and {len(summary['flaky_cases']) - 5} more")
        else:
            print("\n✅ No flaky cases detected")
        
        print("=" * 60)


def compare_versions(
    version_a_results: list[tuple[str, bool]],  # (case_id, passed)
    version_b_results: list[tuple[str, bool]],
    alpha: float = 0.05
) -> dict:
    """
    Compare two versions using statistical significance testing.
    
    Uses chi-squared test for comparing pass rates.
    
    Args:
        version_a_results: List of (case_id, passed) for version A
        version_b_results: List of (case_id, passed) for version B
        alpha: Significance level (default 0.05)
    
    Returns:
        Comparison statistics and significance assessment
    """
    a_passed = sum(1 for _, p in version_a_results if p)
    a_total = len(version_a_results)
    b_passed = sum(1 for _, p in version_b_results if p)
    b_total = len(version_b_results)
    
    a_rate = a_passed / a_total if a_total > 0 else 0
    b_rate = b_passed / b_total if b_total > 0 else 0
    
    # Pooled proportion for chi-squared test
    pooled = (a_passed + b_passed) / (a_total + b_total) if (a_total + b_total) > 0 else 0
    
    # Standard error
    se = math.sqrt(pooled * (1 - pooled) * (1/a_total + 1/b_total)) if a_total > 0 and b_total > 0 else 0
    
    # Z-score
    z_score = (b_rate - a_rate) / se if se > 0 else 0
    
    # Two-tailed p-value approximation (using standard normal)
    # For simplicity, we'll just check against critical values
    z_critical = {0.10: 1.645, 0.05: 1.96, 0.01: 2.576}
    
    is_significant = abs(z_score) > z_critical.get(alpha, 1.96)
    
    return {
        "version_a": {
            "pass_rate": a_rate,
            "passed": a_passed,
            "total": a_total
        },
        "version_b": {
            "pass_rate": b_rate,
            "passed": b_passed,
            "total": b_total
        },
        "difference": b_rate - a_rate,
        "percent_change": ((b_rate - a_rate) / a_rate * 100) if a_rate > 0 else 0,
        "z_score": z_score,
        "is_significant": is_significant,
        "alpha": alpha,
        "conclusion": (
            f"Version B is {'significantly ' if is_significant else ''}{'better' if b_rate > a_rate else 'worse'} "
            f"({'p < {}'.format(alpha) if is_significant else 'not statistically significant'})"
        )
    }


# Example usage
if __name__ == "__main__":
    # Example 1: Reliability analysis with multiple runs
    print("=== Reliability Analysis ===")
    analyzer = ReliabilityAnalyzer()
    
    # Simulate multiple runs for several test cases
    test_runs = [
        # Consistent case - always passes
        RunResult("case_001", 1, True),
        RunResult("case_001", 2, True),
        RunResult("case_001", 3, True),
        
        # Consistent case - always fails  
        RunResult("case_002", 1, False),
        RunResult("case_002", 2, False),
        RunResult("case_002", 3, False),
        
        # Flaky case - inconsistent
        RunResult("case_003", 1, True),
        RunResult("case_003", 2, False),
        RunResult("case_003", 3, True),
        
        # Mostly passes
        RunResult("case_004", 1, True),
        RunResult("case_004", 2, True),
        RunResult("case_004", 3, False),
        
        # Very flaky
        RunResult("case_005", 1, True),
        RunResult("case_005", 2, False),
        RunResult("case_005", 3, False),
    ]
    
    analyzer.add_runs(test_runs)
    analyzer.print_report()
    
    # Example 2: Version comparison
    print("\n=== Version Comparison ===")
    
    # Simulated results from two versions
    version_a = [
        ("test_1", True), ("test_2", True), ("test_3", False),
        ("test_4", True), ("test_5", False), ("test_6", True),
        ("test_7", False), ("test_8", True), ("test_9", True),
        ("test_10", False),
    ] * 10  # 100 cases total
    
    version_b = [
        ("test_1", True), ("test_2", True), ("test_3", True),
        ("test_4", True), ("test_5", False), ("test_6", True),
        ("test_7", True), ("test_8", True), ("test_9", True),
        ("test_10", False),
    ] * 10  # 100 cases total
    
    comparison = compare_versions(version_a, version_b)
    
    print(f"Version A: {comparison['version_a']['pass_rate']:.1%}")
    print(f"Version B: {comparison['version_b']['pass_rate']:.1%}")
    print(f"Difference: {comparison['difference']:+.1%}")
    print(f"Z-score: {comparison['z_score']:.2f}")
    print(f"Conclusion: {comparison['conclusion']}")


---
## Section 18.6: A/B testing agents

In [ ]:
# From: ab_testing.py

# From: AI Agents Book, Chapter 18, Section 18.6
# File: ab_testing.py
# Description: A/B testing framework for comparing agent variants

import hashlib
from dataclasses import dataclass, field
from datetime import datetime
from typing import Callable
import json
from pathlib import Path


@dataclass
class ExperimentResult:
    """Result from a single experiment trial."""
    variant: str
    success: bool
    user_id: str | None = None
    timestamp: str = field(default_factory=lambda: datetime.now().isoformat())
    metadata: dict = field(default_factory=dict)


@dataclass
class ExperimentConfig:
    """Configuration for an A/B experiment."""
    name: str
    hypothesis: str
    primary_metric: str
    variants: list[str] = field(default_factory=lambda: ["A", "B"])
    target_sample_size: int = 500  # Per variant
    
    def to_dict(self) -> dict:
        return {
            "name": self.name,
            "hypothesis": self.hypothesis,
            "primary_metric": self.primary_metric,
            "variants": self.variants,
            "target_sample_size": self.target_sample_size
        }


def get_variant(user_id: str, experiment_name: str, variants: list[str] = ["A", "B"]) -> str:
    """
    Consistently assign a user to a variant.
    Same user always gets the same variant for a given experiment.
    
    Args:
        user_id: Unique identifier for the user
        experiment_name: Name of the experiment
        variants: List of variant names to distribute across
    
    Returns:
        The assigned variant name
    """
    # Create a hash of user_id + experiment_name
    hash_input = f"{user_id}:{experiment_name}"
    hash_value = int(hashlib.md5(hash_input.encode()).hexdigest(), 16)
    
    # Map to variant
    variant_index = hash_value % len(variants)
    return variants[variant_index]


def compare_variants_simple(
    a_successes: int, 
    a_total: int, 
    b_successes: int, 
    b_total: int,
    alpha: float = 0.05
) -> dict:
    """
    Compare two variants using a simple proportion test.
    
    This is a simplified version that doesn't require scipy.
    Uses normal approximation to the binomial.
    
    Args:
        a_successes: Number of successes in variant A
        a_total: Total trials in variant A
        b_successes: Number of successes in variant B
        b_total: Total trials in variant B
        alpha: Significance level (default 0.05)
    
    Returns:
        Dictionary with comparison results
    """
    import math
    
    a_rate = a_successes / a_total if a_total > 0 else 0
    b_rate = b_successes / b_total if b_total > 0 else 0
    
    # Pooled proportion
    pooled = (a_successes + b_successes) / (a_total + b_total) if (a_total + b_total) > 0 else 0
    
    # Standard error
    se = math.sqrt(pooled * (1 - pooled) * (1/a_total + 1/b_total)) if a_total > 0 and b_total > 0 and pooled > 0 and pooled < 1 else 0
    
    # Z-score
    z_score = (b_rate - a_rate) / se if se > 0 else 0
    
    # Critical value for two-tailed test
    z_critical = {0.10: 1.645, 0.05: 1.96, 0.01: 2.576}.get(alpha, 1.96)
    
    significant = abs(z_score) > z_critical
    
    # Determine recommendation
    if significant:
        if b_rate > a_rate:
            recommendation = "B is better - consider rolling out"
        else:
            recommendation = "A is better - keep current version"
    else:
        recommendation = "No significant difference - need more data or keep A"
    
    return {
        "a_rate": a_rate,
        "a_rate_formatted": f"{a_rate:.1%}",
        "b_rate": b_rate,
        "b_rate_formatted": f"{b_rate:.1%}",
        "difference": b_rate - a_rate,
        "difference_formatted": f"{(b_rate - a_rate):+.1%}",
        "relative_improvement": ((b_rate - a_rate) / a_rate * 100) if a_rate > 0 else 0,
        "z_score": z_score,
        "significant": significant,
        "alpha": alpha,
        "recommendation": recommendation
    }


def compare_variants_scipy(
    a_successes: int, 
    a_total: int, 
    b_successes: int, 
    b_total: int
) -> dict:
    """
    Compare two variants using chi-squared test (requires scipy).
    
    Returns whether the difference is statistically significant.
    """
    try:
        from scipy import stats
    except ImportError:
        print("scipy not available, using simple comparison")
        return compare_variants_simple(a_successes, a_total, b_successes, b_total)
    
    # Build contingency table
    table = [
        [a_successes, a_total - a_successes],
        [b_successes, b_total - b_successes]
    ]
    
    chi2, p_value, dof, expected = stats.chi2_contingency(table)
    
    a_rate = a_successes / a_total
    b_rate = b_successes / b_total
    
    return {
        "a_rate": f"{a_rate:.1%}",
        "b_rate": f"{b_rate:.1%}",
        "difference": f"{(b_rate - a_rate):+.1%}",
        "chi2": chi2,
        "p_value": p_value,
        "significant": p_value < 0.05,
        "recommendation": (
            "B is better" if (p_value < 0.05 and b_rate > a_rate) 
            else "A is better" if (p_value < 0.05 and a_rate > b_rate)
            else "No significant difference"
        )
    }


class ABExperiment:
    """
    Manage an A/B testing experiment.
    
    Handles:
    - Variant assignment
    - Result collection
    - Statistical analysis
    - Result persistence
    """
    
    def __init__(self, config: ExperimentConfig):
        self.config = config
        self.results: list[ExperimentResult] = []
        self.start_time = datetime.now()
    
    def assign_variant(self, user_id: str) -> str:
        """Assign a user to a variant."""
        return get_variant(user_id, self.config.name, self.config.variants)
    
    def record_result(self, result: ExperimentResult) -> None:
        """Record a result from the experiment."""
        self.results.append(result)
    
    def record(self, variant: str, success: bool, user_id: str | None = None, **metadata) -> None:
        """Convenience method to record a result."""
        self.record_result(ExperimentResult(
            variant=variant,
            success=success,
            user_id=user_id,
            metadata=metadata
        ))
    
    def get_counts(self) -> dict[str, dict]:
        """Get success/failure counts per variant."""
        counts = {v: {"successes": 0, "failures": 0, "total": 0} 
                  for v in self.config.variants}
        
        for result in self.results:
            if result.variant in counts:
                counts[result.variant]["total"] += 1
                if result.success:
                    counts[result.variant]["successes"] += 1
                else:
                    counts[result.variant]["failures"] += 1
        
        return counts
    
    def progress(self) -> dict:
        """Check experiment progress toward target sample size."""
        counts = self.get_counts()
        
        progress = {}
        for variant, data in counts.items():
            progress[variant] = {
                "current": data["total"],
                "target": self.config.target_sample_size,
                "percent_complete": data["total"] / self.config.target_sample_size * 100
            }
        
        all_complete = all(
            data["total"] >= self.config.target_sample_size 
            for data in counts.values()
        )
        
        return {
            "by_variant": progress,
            "complete": all_complete,
            "total_results": len(self.results)
        }
    
    def analyze(self) -> dict:
        """Analyze experiment results."""
        counts = self.get_counts()
        
        # For two variants, do direct comparison
        if len(self.config.variants) == 2:
            v_a, v_b = self.config.variants
            comparison = compare_variants_simple(
                counts[v_a]["successes"], counts[v_a]["total"],
                counts[v_b]["successes"], counts[v_b]["total"]
            )
        else:
            comparison = None
        
        # Calculate rates for each variant
        rates = {}
        for variant, data in counts.items():
            rate = data["successes"] / data["total"] if data["total"] > 0 else 0
            rates[variant] = {
                "rate": rate,
                "rate_formatted": f"{rate:.1%}",
                "successes": data["successes"],
                "total": data["total"]
            }
        
        return {
            "experiment": self.config.name,
            "hypothesis": self.config.hypothesis,
            "primary_metric": self.config.primary_metric,
            "rates": rates,
            "comparison": comparison,
            "progress": self.progress()
        }
    
    def print_report(self) -> None:
        """Print a formatted experiment report."""
        analysis = self.analyze()
        
        print("\n" + "=" * 60)
        print(f"A/B EXPERIMENT: {analysis['experiment']}")
        print("=" * 60)
        
        print(f"\nHypothesis: {analysis['hypothesis']}")
        print(f"Primary Metric: {analysis['primary_metric']}")
        
        print("\n--- Results by Variant ---")
        for variant, data in analysis['rates'].items():
            bar_len = int(data['rate'] * 30)
            bar = "█" * bar_len + "░" * (30 - bar_len)
            print(f"  {variant}: {data['rate_formatted']} [{bar}]")
            print(f"      ({data['successes']}/{data['total']} successes)")
        
        if analysis['comparison']:
            comp = analysis['comparison']
            print("\n--- Statistical Analysis ---")
            print(f"  Difference: {comp['difference_formatted']}")
            print(f"  Relative improvement: {comp['relative_improvement']:+.1f}%")
            print(f"  Z-score: {comp['z_score']:.2f}")
            print(f"  Significant (α={comp['alpha']}): {'Yes ✓' if comp['significant'] else 'No'}")
            print(f"\n  📊 Recommendation: {comp['recommendation']}")
        
        prog = analysis['progress']
        print("\n--- Progress ---")
        for variant, data in prog['by_variant'].items():
            print(f"  {variant}: {data['current']}/{data['target']} ({data['percent_complete']:.0f}%)")
        
        status = "✅ Complete" if prog['complete'] else "⏳ In Progress"
        print(f"\n  Status: {status}")
        print("=" * 60)
    
    def save(self, path: str) -> None:
        """Save experiment state to JSON."""
        data = {
            "config": self.config.to_dict(),
            "start_time": self.start_time.isoformat(),
            "results": [
                {
                    "variant": r.variant,
                    "success": r.success,
                    "user_id": r.user_id,
                    "timestamp": r.timestamp,
                    "metadata": r.metadata
                }
                for r in self.results
            ]
        }
        
        with open(path, 'w') as f:
            json.dump(data, f, indent=2)
        
        print(f"Experiment saved to {path}")
    
    @classmethod
    def load(cls, path: str) -> "ABExperiment":
        """Load experiment from JSON."""
        with open(path, 'r') as f:
            data = json.load(f)
        
        config = ExperimentConfig(**data["config"])
        experiment = cls(config)
        experiment.start_time = datetime.fromisoformat(data["start_time"])
        
        for r in data["results"]:
            experiment.results.append(ExperimentResult(
                variant=r["variant"],
                success=r["success"],
                user_id=r["user_id"],
                timestamp=r["timestamp"],
                metadata=r["metadata"]
            ))
        
        return experiment


# Example usage
if __name__ == "__main__":
    import random
    
    # Create an experiment
    config = ExperimentConfig(
        name="prompt_v2_test",
        hypothesis="A more detailed system prompt will increase task completion from 75% to 85%",
        primary_metric="task_completion_rate",
        target_sample_size=100  # Small for demo
    )
    
    experiment = ABExperiment(config)
    
    # Simulate running the experiment
    print("Simulating experiment...")
    
    # Simulate 200 users
    for i in range(200):
        user_id = f"user_{i}"
        variant = experiment.assign_variant(user_id)
        
        # Simulate different success rates
        # A has 75% success, B has 85% success
        if variant == "A":
            success = random.random() < 0.75
        else:
            success = random.random() < 0.85
        
        experiment.record(variant=variant, success=success, user_id=user_id)
    
    # Print results
    experiment.print_report()
    
    # Demonstrate variant assignment consistency
    print("\n--- Variant Assignment Demo ---")
    test_user = "user_12345"
    for _ in range(5):
        v = get_variant(test_user, "prompt_v2_test")
        print(f"  {test_user} -> {v}")
    print("  (Same user always gets same variant)")


---
## Section 18.7: Continuous improvement workflows

In [ ]:
# From: continuous_improvement.py

# From: AI Agents Book, Chapter 18, Section 18.7
# File: continuous_improvement.py
# Description: Tools for continuous improvement workflows - scheduled evaluation, monitoring, reporting

import json
from datetime import datetime, timedelta
from pathlib import Path
from dataclasses import dataclass, field
from typing import Callable
from collections import defaultdict


@dataclass
class EvaluationRun:
    """Record of a single evaluation run."""
    timestamp: str
    version: str
    overall_score: float
    pass_rate: float
    total_cases: int
    by_category: dict = field(default_factory=dict)
    failures: list = field(default_factory=list)
    metadata: dict = field(default_factory=dict)


class QualityTracker:
    """
    Track quality metrics over time for continuous improvement.
    
    Implements the measurement and analysis phases of the improvement cycle.
    """
    
    def __init__(self, storage_path: str = "quality_history.json"):
        self.storage_path = Path(storage_path)
        self.history: list[EvaluationRun] = []
        self._load()
    
    def _load(self) -> None:
        """Load history from storage."""
        if self.storage_path.exists():
            with open(self.storage_path, 'r') as f:
                data = json.load(f)
                self.history = [EvaluationRun(**run) for run in data]
    
    def _save(self) -> None:
        """Save history to storage."""
        data = [
            {
                "timestamp": run.timestamp,
                "version": run.version,
                "overall_score": run.overall_score,
                "pass_rate": run.pass_rate,
                "total_cases": run.total_cases,
                "by_category": run.by_category,
                "failures": run.failures,
                "metadata": run.metadata
            }
            for run in self.history
        ]
        with open(self.storage_path, 'w') as f:
            json.dump(data, f, indent=2)
    
    def record_run(self, run: EvaluationRun) -> None:
        """Record an evaluation run."""
        self.history.append(run)
        self._save()
    
    def record(
        self,
        version: str,
        overall_score: float,
        pass_rate: float,
        total_cases: int,
        by_category: dict | None = None,
        failures: list | None = None,
        **metadata
    ) -> EvaluationRun:
        """Convenience method to record a run."""
        run = EvaluationRun(
            timestamp=datetime.now().isoformat(),
            version=version,
            overall_score=overall_score,
            pass_rate=pass_rate,
            total_cases=total_cases,
            by_category=by_category or {},
            failures=failures or [],
            metadata=metadata
        )
        self.record_run(run)
        return run
    
    def get_latest(self) -> EvaluationRun | None:
        """Get the most recent evaluation run."""
        return self.history[-1] if self.history else None
    
    def get_trend(self, metric: str = "pass_rate", days: int = 30) -> list[tuple[str, float]]:
        """Get trend data for a metric over time."""
        cutoff = datetime.now() - timedelta(days=days)
        
        trend = []
        for run in self.history:
            run_time = datetime.fromisoformat(run.timestamp)
            if run_time >= cutoff:
                value = getattr(run, metric, None)
                if value is not None:
                    trend.append((run.timestamp[:10], value))
        
        return trend
    
    def compare_to_baseline(self, baseline_version: str) -> dict | None:
        """Compare latest results to a baseline version."""
        # Find baseline
        baseline = None
        for run in self.history:
            if run.version == baseline_version:
                baseline = run
                break
        
        if not baseline:
            return None
        
        latest = self.get_latest()
        if not latest:
            return None
        
        return {
            "baseline": {
                "version": baseline.version,
                "pass_rate": baseline.pass_rate,
                "overall_score": baseline.overall_score
            },
            "current": {
                "version": latest.version,
                "pass_rate": latest.pass_rate,
                "overall_score": latest.overall_score
            },
            "change": {
                "pass_rate": latest.pass_rate - baseline.pass_rate,
                "overall_score": latest.overall_score - baseline.overall_score
            },
            "improved": latest.pass_rate > baseline.pass_rate
        }
    
    def detect_regression(self, threshold: float = 0.05) -> dict | None:
        """
        Detect if recent performance has regressed.
        
        Args:
            threshold: Minimum drop to consider a regression
        
        Returns:
            Regression info if detected, None otherwise
        """
        if len(self.history) < 2:
            return None
        
        # Compare last run to average of previous 5
        recent = self.history[-1]
        previous = self.history[-6:-1] if len(self.history) >= 6 else self.history[:-1]
        
        avg_pass_rate = sum(r.pass_rate for r in previous) / len(previous)
        
        drop = avg_pass_rate - recent.pass_rate
        
        if drop > threshold:
            return {
                "detected": True,
                "current_pass_rate": recent.pass_rate,
                "average_pass_rate": avg_pass_rate,
                "drop": drop,
                "version": recent.version,
                "message": f"⚠️ Regression detected: {drop:.1%} drop from recent average"
            }
        
        return {"detected": False}


class QualityReporter:
    """Generate quality reports from tracking data."""
    
    def __init__(self, tracker: QualityTracker):
        self.tracker = tracker
    
    def weekly_report(self) -> str:
        """Generate a weekly quality report."""
        # Get runs from last 7 days
        cutoff = datetime.now() - timedelta(days=7)
        recent_runs = [
            run for run in self.tracker.history
            if datetime.fromisoformat(run.timestamp) >= cutoff
        ]
        
        if not recent_runs:
            return "No evaluation runs in the past week."
        
        # Calculate statistics
        pass_rates = [r.pass_rate for r in recent_runs]
        scores = [r.overall_score for r in recent_runs]
        
        avg_pass_rate = sum(pass_rates) / len(pass_rates)
        avg_score = sum(scores) / len(scores)
        latest = recent_runs[-1]
        
        # Collect all failures
        all_failures = []
        for run in recent_runs:
            all_failures.extend(run.failures)
        
        # Count failure frequency
        failure_counts = defaultdict(int)
        for failure in all_failures:
            failure_id = failure.get('case_id', str(failure))
            failure_counts[failure_id] += 1
        
        # Build report
        lines = [
            "=" * 60,
            "WEEKLY QUALITY REPORT",
            f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}",
            "=" * 60,
            "",
            "📊 SUMMARY",
            f"  Evaluation runs: {len(recent_runs)}",
            f"  Average pass rate: {avg_pass_rate:.1%}",
            f"  Average score: {avg_score:.2f}",
            "",
            "📈 LATEST RUN",
            f"  Version: {latest.version}",
            f"  Pass rate: {latest.pass_rate:.1%}",
            f"  Score: {latest.overall_score:.2f}",
            f"  Cases: {latest.total_cases}",
        ]
        
        # Category breakdown
        if latest.by_category:
            lines.append("")
            lines.append("📁 BY CATEGORY")
            for cat, stats in latest.by_category.items():
                rate = stats.get('pass_rate', stats.get('accuracy', 0))
                lines.append(f"  {cat}: {rate:.1%}")
        
        # Top failures
        if failure_counts:
            lines.append("")
            lines.append("❌ FREQUENT FAILURES")
            top_failures = sorted(failure_counts.items(), key=lambda x: -x[1])[:5]
            for failure_id, count in top_failures:
                lines.append(f"  {failure_id}: {count} times")
        
        # Trend
        trend = self.tracker.get_trend("pass_rate", days=7)
        if len(trend) >= 2:
            first_rate = trend[0][1]
            last_rate = trend[-1][1]
            change = last_rate - first_rate
            
            lines.append("")
            lines.append("📉 TREND")
            if change > 0.01:
                lines.append(f"  ✅ Improving: +{change:.1%} over the week")
            elif change < -0.01:
                lines.append(f"  ⚠️ Declining: {change:.1%} over the week")
            else:
                lines.append(f"  ➡️ Stable: {change:+.1%} over the week")
        
        # Regression check
        regression = self.tracker.detect_regression()
        if regression and regression.get("detected"):
            lines.append("")
            lines.append("🚨 ALERT")
            lines.append(f"  {regression['message']}")
        
        lines.append("")
        lines.append("=" * 60)
        
        return "\n".join(lines)
    
    def improvement_priorities(self) -> list[dict]:
        """
        Identify improvement priorities based on recent data.
        
        Returns list sorted by estimated impact.
        """
        latest = self.tracker.get_latest()
        if not latest:
            return []
        
        priorities = []
        
        # Analyze category performance
        for cat, stats in latest.by_category.items():
            rate = stats.get('pass_rate', stats.get('accuracy', 0))
            count = stats.get('count', stats.get('total', 0))
            
            # Impact = volume * failure rate
            failure_rate = 1 - rate
            impact = count * failure_rate
            
            if failure_rate > 0.1:  # Only flag categories with >10% failure
                priorities.append({
                    "category": cat,
                    "pass_rate": rate,
                    "failure_rate": failure_rate,
                    "volume": count,
                    "estimated_impact": impact,
                    "recommendation": f"Improve {cat} handling (currently {rate:.0%} pass rate)"
                })
        
        # Sort by impact
        priorities.sort(key=lambda x: -x["estimated_impact"])
        
        return priorities


class ScheduledEvaluator:
    """
    Run evaluations on a schedule.
    
    In production, this would be triggered by cron or a task scheduler.
    This class provides the logic for what to run.
    """
    
    def __init__(
        self,
        tracker: QualityTracker,
        evaluate_fn: Callable[[], dict],
        version: str = "current"
    ):
        """
        Args:
            tracker: QualityTracker to record results
            evaluate_fn: Function that runs evaluation and returns results dict
                        Expected format: {"pass_rate": float, "overall_score": float, 
                                         "total_cases": int, "by_category": dict, "failures": list}
            version: Version identifier for this agent
        """
        self.tracker = tracker
        self.evaluate_fn = evaluate_fn
        self.version = version
    
    def run_evaluation(self) -> EvaluationRun:
        """Run evaluation and record results."""
        print(f"🔄 Running evaluation for version {self.version}...")
        
        results = self.evaluate_fn()
        
        run = self.tracker.record(
            version=self.version,
            overall_score=results.get("overall_score", 0),
            pass_rate=results.get("pass_rate", 0),
            total_cases=results.get("total_cases", 0),
            by_category=results.get("by_category", {}),
            failures=results.get("failures", [])
        )
        
        print(f"✅ Evaluation complete: {run.pass_rate:.1%} pass rate")
        
        # Check for regression
        regression = self.tracker.detect_regression()
        if regression and regression.get("detected"):
            print(f"🚨 {regression['message']}")
        
        return run
    
    def should_run(self, schedule: str = "daily") -> bool:
        """
        Check if evaluation should run based on schedule.
        
        Args:
            schedule: "daily", "weekly", or "hourly"
        """
        latest = self.tracker.get_latest()
        if not latest:
            return True  # Never run before
        
        last_run = datetime.fromisoformat(latest.timestamp)
        now = datetime.now()
        
        if schedule == "hourly":
            return (now - last_run) >= timedelta(hours=1)
        elif schedule == "daily":
            return (now - last_run) >= timedelta(days=1)
        elif schedule == "weekly":
            return (now - last_run) >= timedelta(weeks=1)
        
        return True


# Example usage
if __name__ == "__main__":
    import random
    
    # Initialize tracker
    tracker = QualityTracker("demo_quality_history.json")
    
    # Simulate some historical evaluation runs
    print("Simulating evaluation history...")
    
    categories = ["password_reset", "order_status", "refunds", "technical"]
    
    for i in range(10):
        # Simulate gradually improving performance
        base_rate = 0.75 + (i * 0.015) + random.uniform(-0.03, 0.03)
        
        by_category = {}
        for cat in categories:
            cat_rate = base_rate + random.uniform(-0.1, 0.1)
            cat_rate = max(0.5, min(1.0, cat_rate))
            by_category[cat] = {
                "pass_rate": cat_rate,
                "count": random.randint(20, 50)
            }
        
        failures = []
        if random.random() > base_rate:
            failures.append({"case_id": f"case_{random.randint(1, 100)}"})
        
        tracker.record(
            version=f"v1.{i}",
            overall_score=base_rate * 5,  # Score out of 5
            pass_rate=base_rate,
            total_cases=100,
            by_category=by_category,
            failures=failures
        )
    
    # Generate reports
    reporter = QualityReporter(tracker)
    
    print("\n" + reporter.weekly_report())
    
    # Show improvement priorities
    print("\n📋 IMPROVEMENT PRIORITIES")
    print("-" * 40)
    priorities = reporter.improvement_priorities()
    for i, p in enumerate(priorities[:3], 1):
        print(f"{i}. {p['recommendation']}")
        print(f"   Impact score: {p['estimated_impact']:.1f}")
    
    # Compare to baseline
    print("\n📊 COMPARISON TO BASELINE")
    print("-" * 40)
    comparison = tracker.compare_to_baseline("v1.0")
    if comparison:
        print(f"Baseline (v1.0): {comparison['baseline']['pass_rate']:.1%}")
        print(f"Current: {comparison['current']['pass_rate']:.1%}")
        print(f"Change: {comparison['change']['pass_rate']:+.1%}")
        status = "✅ Improved!" if comparison['improved'] else "⚠️ Declined"
        print(f"Status: {status}")
    
    # Clean up demo file
    Path("demo_quality_history.json").unlink(missing_ok=True)


---
## Next Steps

- Check your answers in **chapter_18_testing_solutions.ipynb**
- Proceed to **Chapter 19**